In [2]:
%env KERAS_BACKEND=torch

from typing import TypedDict, Unpack
import functools as _functools_

from more_itertools import sliding_window 
from keras import Model, Sequential, layers, KerasTensor

# TODO use keras!!!!
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
####


class PVPredictor:

    class Input(TypedDict):
        out_w: KerasTensor
        r"""The previous photovoltaic power outputs (W)"""

    class Output(TypedDict):
        out_w: KerasTensor
        r"""The next photovoltaic power output (W)"""

    def _make_model(self, **model_kwds):
        # TODO rm
        
        inputs: PVPredictor.Input = {
            'out_w': layers.Input(shape=(None, )),
            # 'drytemp_c': layers.Input(shape=(None, 1)),
            # 'airpress_pa': layers.Input(shape=(None, 1)),
            # 'windspeed_mps': layers.Input(shape=(None, 1)),
            # 'winddir_deg': layers.Input(shape=(None, 1)),
            # 'diffsolar_wpsqm': layers.Input(shape=(None, )),
            'directsolar_wpsqm': layers.Input(shape=(None, )),
            # 'solazim_deg': layers.Input(shape=(None, 1)),
            # 'solalti_deg': layers.Input(shape=(None, 1)),
        }

        concatenated_inputs = layers.Concatenate(axis=-1)(
            list(layers.Reshape((-1, 1))(x) for x in inputs.values())
        )
        body = Sequential([
            layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
            layers.MaxPooling1D(pool_size=2),
            layers.Dropout(0.5),
            layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
            layers.Dropout(0.5),
            layers.Bidirectional(layers.LSTM(100, return_sequences=False)),
            layers.Dropout(0.5),
        ])
        outputs: PVPredictor.Output = {
            'out_w': layers.Lambda(lambda x: x[..., 0])(
                layers.Dense(1)(body(concatenated_inputs))
            ),
        }

        model = Model(inputs=inputs, outputs=outputs, **model_kwds)
        model.compile(optimizer='adam', loss='mse')
        return model

    def __init__(self):
        self.model = self._make_model()

    @_functools_.cached_property
    def experience(self) -> list[tuple[Input, Output]]:
        return list()
    
    def train(self, **fit_kwds):
        truths = [
            (input_curr, {'out_w': input_next['out_w']})
            for (input_curr, _), (input_next, _) in 
            sliding_window(self.experience, n=2)
        ]

        if len(truths) == 0:
            return
        return self.model.fit(
            (x for x in truths),
            shuffle=False,
            **fit_kwds,
        )
    
    def __call__(self, inputs: Input) -> Output:
        output_pred = self.model(inputs, training=False)
        self.experience.append((inputs, output_pred))
        return output_pred

env: KERAS_BACKEND=torch


In [3]:
PVPredictor().model(
    layers.Input(shape=(20, 1))
)

2024-11-03 05:15:08.536164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730610908.547564   51439 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730610908.550923   51439 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 05:15:08.563455: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/keras/src/models/functional.py:106: UserWarning: When p

ValueError: Layer "functional_1" expects 2 input(s), but it received 1 input tensors. Inputs received: [<KerasTensor shape=(None, 20, 1), dtype=float32, sparse=False, name=keras_tensor_18>]

In [ ]:
PVPredictor()._model(np.zeros((1, 5, 1)))

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/keras/src/models/functional.py:106: UserWarning: When providing `inputs` as a dict, all keys in the dict must match the names of the corresponding tensors. Received key 'out_w' mapping to value <KerasTensor shape=(None, None), dtype=float32, sparse=False, name=keras_tensor_484> which has name 'keras_tensor_484'. Change the tensor name to 'out_w' (via `Input(..., name='out_w')`)
  warnings.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/keras/src/models/functional.py:106: UserWarning: When providing `inputs` as a dict, all keys in the dict must match the names of the corresponding tensors. Received key 'directsolar_wpsqm' mapping to value <KerasTensor shape=(None, None), dtype=float32, sparse=False, name=keras_tensor_485> which has name 'keras_tensor_485'. Change the tensor name to 'directsolar_wpsqm' (via `Input(..., name='directsolar_wpsqm')`)
  warnings.warn(


tensor([[0.]], device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
np.array([[[1], [2], [3]]]).shape

(1, 3, 1)

In [ ]:
import numpy as np

# TODO mv doctest
# PVPredictor._make_model()({
#     'out_w': np.array([[1, 2, 3, 4]]),
# })

In [ ]:
# predictor = PVPredictor()

# for _ in range(20):
#     predictor({
#         'out_w': np.array([[1, 2, 3, 4, 7]]),
#     })
#     if len(predictor.experience) >= 10:
#         predictor.train()
#         predictor.experience.clear()

In [ ]:
from collections import deque

class PVTrackingPredictor:
    class Observation(TypedDict):
        out_w: float
        r"""The current photovoltaic power output (W)"""

    class Result(TypedDict):
        out_w: float
        r"""The next photovoltaic power output (W)"""

    def __init__(
        self, 
        lookback: int | None = None, 
        **predictor_kwds,
    ):
        self.predictor = PVPredictor(**predictor_kwds)
        self.history = deque(maxlen=lookback)

    def __call__(
        self, 
        observation: Observation, 
        lookback: int | None = None,
        experience_len: int = 1,
        **fit_kwds,
    ) -> Result | None:
        self.history.append(observation)
        if len(self.history) < (
            lookback 
            if lookback is not None else 
            # TODO what if this is None as well??
            self.history.maxlen
        ):
            return None

        if len(self.predictor.experience) >= experience_len:
            self.predictor.train(**fit_kwds)
            self.predictor.experience.clear()
            
        predictor_inputs: PVPredictor.Input = {
            k: np.array([[x[k] for x in self.history]])
            for k in ('out_w', 'directsolar_wpsqm')
        }
        res = self.predictor(predictor_inputs)        
        # TODO proc res
        return {
            'out_w': res['out_w'][0],
        }

In [ ]:
# tracking_predictor = PVTrackingPredictor(lookback=10)

# for out_w in range(10):
#     print(tracking_predictor({'out_w': out_w}, experience_len=100))

In [ ]:
import pandas as pd
df = pd.read_csv('filtered_solar_data.csv')
df

,clock,pv,Drybulb_Temperature,Air_Pressure,Wind_Speed,Wind_Direction,Diffuse_Solar_Radiation,Direct_Solar_Radiation,Solar_Azimuth_Angle,Solar_Altitude_Angle,Clock
0,1999-07-01 00:10:00+00:00,0.0,27.633333,100700.000000,0.583333,211.666667,0.0,0.0,28.851525,-61.751482,07-01 00:10:00
1,1999-07-01 00:20:00+00:00,0.0,27.266667,100700.000000,0.466667,233.333333,0.0,0.0,24.534594,-62.875105,07-01 00:20:00
2,1999-07-01 00:30:00+00:00,0.0,26.900000,100700.000000,0.350000,255.000000,0.0,0.0,19.862324,-63.820269,07-01 00:30:00
3,1999-07-01 00:40:00+00:00,0.0,26.533333,100700.000000,0.233333,276.666667,0.0,0.0,14.867243,-64.566991,07-01 00:40:00
4,1999-07-01 00:50:00+00:00,0.0,26.166667,100700.000000,0.116667,298.333333,0.0,0.0,9.606702,-65.097355,07-01 00:50:00
...,...,...,...,...,...,...,...,...,...,...,...
4315,1999-07-30 23:20:00+00:00,0.0,26.233333,100533.333333,1.900000,140.000000,0.0,0.0,307.787278,-56.248959,07-30 23:20:00
4316,1999-07-30 23:30:00+00:00,0.0,26.000000,100550.000000,1.800000,140.000000,0.0,0.0,310.292169,-58.190742,07-30 23:30:00
4317,1999-07-30 23:40:00+00:00,0.0,25.766667,100566.666667,1.700000,140.000000,0.0,0.0,313.134014,-60.057104,07-30 23:40:00
4318,1999-07-30 23:50:00+00:00,0.0,25.533333,100583.333333,1.600000,140.000000,0.0,0.0,316.361694,-61.832944,07-30 23:50:00


In [ ]:
import tensorflow as _tensorflow_

layers.Normalization().adapt

_tensorflow_.data.Dataset.from_tensor_slices({'a': np.array([[1], [2]])})
_tensorflow_.data.Dataset.from_tensor_slices({'x': {'a': np.array([[1], [2]])}})

<_TensorSliceDataset element_spec={'x': {'a': TensorSpec(shape=(1,), dtype=tf.int64, name=None)}}>

In [ ]:
#layers.Normalization().adapt({'x': {'a': np.array([[1], [2]])}})
#.adapt({'a': [[1], [2]]})

In [ ]:
import pandas as pd
df = pd.read_csv('filtered_solar_data.csv')
#features = ['pv']
#X = df[features]#.values

tracking_predictor = PVTrackingPredictor(lookback=1_000)

for out_w, directsolar_wpsqm in zip(df['pv'], df['Direct_Solar_Radiation']):
    (tracking_predictor(
        {'out_w': out_w, 'directsolar_wpsqm': directsolar_wpsqm}, 
        experience_len=10,
        epochs=10,
    ))

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/keras/src/models/functional.py:106: UserWarning: When providing `inputs` as a dict, all keys in the dict must match the names of the corresponding tensors. Received key 'out_w' mapping to value <KerasTensor shape=(None, None), dtype=float32, sparse=False, name=keras_tensor_18> which has name 'keras_tensor_18'. Change the tensor name to 'out_w' (via `Input(..., name='out_w')`)
  warnings.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/keras/src/models/functional.py:106: UserWarning: When providing `inputs` as a dict, all keys in the dict must match the names of the corresponding tensors. Received key 'directsolar_wpsqm' mapping to value <KerasTensor shape=(None, None), dtype=float32, sparse=False, name=keras_tensor_19> which has name 'keras_tensor_19'. Change the tensor name to 'directsolar_wpsqm' (via `Input(..., name='directsolar_wpsqm')`)
  warnings.warn(


Epoch 1/10


/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/keras/src/backend/torch/numpy.py:1143: UserWarning: No PYTORCH_KERNEL_CACHE_PATH or HOME environment variable set! This disables kernel caching. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1426.)
  return cast(torch.prod(x, dtype=to_torch_dtype(compute_dtype)), dtype)


8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 25790714478592.0000
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 164us/step
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 136us/step
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 97us/step
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 88us/step
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 85us/step
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 103us/step
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 105us/step
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 98us/step
Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 25790693507072.0000
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 73us/step
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 83us/step
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 65us/step
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 64us/step
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 67us/step
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 73us/step
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 61us/step
Epoch 10/10
9/9 ━━━━

KeyboardInterrupt: 

In [ ]:
%env KERAS_BACKEND=tensorflow

from typing import TypedDict, Unpack
import functools as _functools_

from more_itertools import sliding_window 
from keras import Model, Sequential, layers, KerasTensor

import numpy as np


class PVPredictorModel:
    @staticmethod
    def _invert_normalizer(normalizer: layers.Normalization):
        return layers.Normalization(
            axis=normalizer.axis, 
            invert=True,
            **(
                dict(
                    mean=normalizer.mean, 
                    variance=normalizer.variance,                 
                ) 
                if normalizer.built else 
                dict()
            ),
        )

    @_functools_.cached_property
    def _input_normalizer(self):
        return layers.Normalization(
            axis=-1,
        )
    
    @_functools_.cached_property
    def _output_normalizer(self):
        return layers.Normalization(
            axis=-1,
        )
    
    @_functools_.cached_property
    def _model(self):
        model = Sequential([
            layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
            layers.MaxPooling1D(pool_size=2),
            layers.Dropout(0.5),
            layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
            layers.Dropout(0.5),
            layers.Bidirectional(layers.LSTM(100, return_sequences=False)),
            layers.Dropout(0.5),
            layers.Dense(1),
        ])
        model.compile(optimizer='adam', loss='mse')
        return model
    
    def fit(self, inputs, outputs, **kwargs):
        self._input_normalizer.adapt(inputs)
        self._output_normalizer.adapt(outputs)

        return self._model.fit(
            x=self._input_normalizer(inputs),
            y=self._output_normalizer(outputs),
            **kwargs,
        )
    
    def predict(self, inputs, **kwargs): 
        output = self._model.predict(
            self._input_normalizer(inputs),
            **kwargs,
        )

        denormalizer = self._invert_normalizer(
            self._output_normalizer
        )
        return denormalizer(output)



from collections import deque

class PVPredictor:
    class Observation(TypedDict):
        out_w: float
        r"""The current photovoltaic power output (W)"""

        @classmethod
        def to_array(cls, obs):
            if isinstance(obs, dict):
                return np.array(list(obs.values()))
            return np.array([cls.to_array(x) for x in obs])

    class Result(TypedDict):
        out_w: float
        r"""The next photovoltaic power output (W)"""

    Batch = list

    Truth = tuple[list[Observation], Result]

    def __init__(self, lookback: int | None = None):
        self.model = PVPredictorModel()
        self.observations = deque[self.Observation](maxlen=lookback)
    
    def fit(self, data_batch: Batch[Truth], **kwargs):
        inputs = np.array([
            [list(obs.values()) for obs in obss] 
            for obss, _ in data_batch
        ])
        outputs = np.array([
            list(res.values()) 
            for _, res in data_batch
        ])

        return self.model.fit(
            inputs, outputs,
            shuffle=False,
            **kwargs,
        )

    def predict(self, data_batch: Batch[list[Observation]], **kwargs) -> Batch[Result]:
        output_batch = self.model.predict(
            np.array([[list(obs.values()) for obs in batch] for batch in data_batch]),
            **kwargs,
        )
        res = [{'out_w': output[0]} for output in output_batch]
        return res

    @_functools_.cached_property
    def experience(self) -> list[Truth]:
        return list()
    
    @property
    def true_experience(self) -> list[Truth]:
        return [
            (obs_curr, {'out_w': obs_next[0]['out_w']})
            for (obs_curr, _), (obs_next, _) in 
            sliding_window(self.experience, n=2)
        ]
    
    def __call__(
        self, 
        observation: Observation,
        lookback: int | None = None,
        experience_len: int = 1,
        fit_options: dict = dict(),
        predict_options: dict = dict(),
    ) -> Result:
        self.observations.append(observation)
        if len(self.observations) < (
            lookback 
            if lookback is not None else 
            # TODO what if this is None as well??
            self.observations.maxlen
        ):
            return None
            
        [res, ] = self.predict([list(self.observations)], **predict_options)
        self.experience.append((list(self.observations), res))
        if len(self.experience) >= experience_len:
            self.fit(list(self.true_experience), **fit_options)
            self.experience.clear()        
        return res


env: KERAS_BACKEND=tensorflow


2024-11-03 05:20:33.063251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730611233.074333   52488 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730611233.077625   52488 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 05:20:33.090146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

import pandas as pd
import plotly.graph_objects
df = pd.read_csv('filtered_solar_data.csv')

predictor = PVPredictor(lookback=2_000)

fig = plotly.graph_objects.FigureWidget()
fig.add_scatter(y=[], name='truth')
fig.add_scatter(y=[], name='predicted')
display(fig)

for out_w, directsolar_wpsqm, solarangle in zip(df['pv'], df['Direct_Solar_Radiation'], df['Solar_Azimuth_Angle']):
    predicted = predictor(
        {'out_w': out_w, 'directsolar_wpsqm': directsolar_wpsqm, 'solarangle': solarangle}, 
        experience_len=100,
        fit_options=dict(epochs=16),
        predict_options=dict(verbose=0),
    )

    if predicted is not None:
        fig.data[0].y += (out_w, )
        fig.data[1].y += (predicted['out_w'].cpu().numpy() if predicted is not None else None, )

FigureWidget({
    'data': [{'name': 'truth', 'type': 'scatter', 'uid': 'd84f5c62-a7d2-4e67-b9ef-09084da2c413', 'y': []},
             {'name': 'predicted', 'type': 'scatter', 'uid': '7fbeada8-8233-4a10-9c1d-394f5b876728', 'y': []}],
    'layout': {'template': '...'}
})

W0000 00:00:1730611235.618718   52488 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Instructions for updating:
Use tf.identity with explicit device placement instead.


KeyboardInterrupt: 

In [2]:
import tensorflow as tf
tf.executing_eagerly()

True

In [2]:
from predict_next import PVPredictor

predictor = PVPredictor(lookback=100)


for _ in range(1000):
    predictor(
        {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003,},
        experience_len=10,
        #fit_options=dict(epochs=16),
        #predict_options=dict(verbose=0),
    )

predict [[[ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  

predict [[[ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  [ 0.    28.386]
  

KeyboardInterrupt: 

In [1]:
import keras
keras.config.disable_traceback_filtering()

from predict_next import PVPredictor

predictor = PVPredictor(lookback=10)


for _ in range(20):
    print(
        predictor(
            {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
            experience_len=10,
            #fit_options=dict(epochs=16),
            #predict_options=dict(verbose=0),
        )
    )

2024-11-03 14:05:36.708251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730642736.719373  285829 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730642736.722768  285829 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 14:05:36.735642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


W0000 00:00:1730642738.116597  285829 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - loss: 3.7691e-30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
{'out_w': 2.0499834e-12}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
{'out_w': 2.0499834e-12}


In [6]:
predictor.observations

deque([{'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123},
       {'out_w': 0.0, 'Drybulb_Temperature': 28.386000000000003, 'a': 123}],
      maxlen=10)

In [3]:
import sklearn.preprocessing

sklearn.preprocessing.StandardScaler().fit_transform([1, 2, 3])

ValueError: Expected 2D array, got 1D array instead:
array=[1. 2. 3.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.